## Tools

#### Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:

##### A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
##### A function or coroutine to execute.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

c:\Users\ujami\Downloads\PHD_thesis\langchain_learning\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


AIMessage(content="<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know about parrots. They're birds known for mimicking sounds, including human speech. But why do they do that? Maybe it's about communication. I remember hearing that some birds use mimicry to interact with others or for survival.\n\nFirst, I should consider their natural behavior. Parrots are social animals, right? They live in flocks in the wild. Maybe talking helps them bond with their flock. But how does mimicking sounds help them in the wild? Maybe they imitate other birds or animals to blend in or avoid predators. Wait, but in captivity, they talk to humans. So maybe it's a learned behavior based on their environment.\n\nI also think about how they learn. Parrots have a part of the brain called the song system, which is involved in learning and producing sounds. They might imitate humans because they're exposed to those sounds and associate them with positive experiences l

In [2]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [3]:
model_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000023EAA26D6A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023EAA26E120>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the weather at a location', 'parameters': {'properties': {'location': {'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [4]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which is Boston here. I'll call the function with location set to Boston. Make sure the JSON is correctly formatted with the name and arguments.\n", 'tool_calls': [{'id': 'gk2ygze6t', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 154, 'total_tokens': 240, 'completion_time': 0.153530945, 'completion_tokens_details': {'reasoning_tokens': 62}, 'prompt_time': 0.007295498, 'prompt_tokens_details': None, 'queue_time': 0.264651294, 'total_time': 0.160826443}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019b5b9f-351a-7072-a

## Tool Execution Loop

In [5]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)

The weather in Boston is sunny right now. A perfect day to enjoy outdoor activities! 😊


In [6]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. I need to use the get_weather function. Let me check the function parameters. The required parameter is location, which should be a string. Boston is the location here. So I should call get_weather with location set to "Boston". Make sure the JSON is correctly formatted with the name and arguments. No other parameters are needed. Let me structure the tool_call accordingly.\n', 'tool_calls': [{'id': '87vf75w2k', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 153, 'total_tokens': 263, 'completion_time': 0.162458826, 'completion_tokens_details': {'reasoning_tokens': 86}, 'prompt_time': 0.006161554, 'prompt_tokens_details': None, 'queue_time': 0.090472652, 'total_time': 0.16862038}, 'mod